In [8]:
from jax.config import config
config.update("jax_enable_x64", True)

import jax.numpy as jnp
from jax import jit, lax, vmap, jacobian
import jax.scipy.linalg as jsl

import jax

import numpy as np
from read_traj_v2 import *

import scipy.optimize

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'torch'

In [9]:
!pip3 install torch torchvision torchaudio

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 18.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 131.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 93.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 35.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 46.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 21.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 56.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 58.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 217.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
import torch
import torch.nn as nn

In [18]:
mol = 'heh+'
basis = 'sto-3g'
init = 'hf'
td = 'rt-tdexx'

dt = 0.08268/10

inpath = './correctdata'
inpath2 = './mydata/'
outpath = './'
saved_model_dir = './models/'

trnfrq = jnp.array(np.load('./tdcis_data/tdcis_trnfreq_dt0.008268.npz'))
trnamp = jnp.array(np.load('./tdcis_data/tdcis_trnamp_dt0.008268.npz'))
trntme = jnp.array(np.load('./tdcis_data/tdcis_trntme_dt0.008268.npz'))
# load important static matrices
a = traj_data(inpath, mol, basis, init, td, '001',
                init_cond_file=f'ke+en+overlap+ee_twoe+dip_casscf22_{mol}_{basis}.npz',
                )
a.assign_init_data()
drc = a.drc
xmat, didat, kinmat, enmat = a.xmat, a.didat, a.kinmat, a.enmat
X = xmat


	reading from "./correctdata/ke+en+overlap+ee_twoe+dip_casscf22_heh+_sto-3g.npz"

can only concatenate str (not "NoneType") to str
True is not in list


In [19]:
trnden = np.array(np.load('./tdcis_data/tdcis_dt0.0082680.npz'))
print(trnden.shape)

(301, 10000, 2, 2)


In [20]:
trntimecoeffs = np.array(np.load('./tdcis_data/time_coeffs_tdcis_dt0.008268.npz'))
print(trntimecoeffs.shape)

(300, 10000, 2)


In [72]:
torch.set_default_dtype(torch.float64)
input_size = 4
output_size = 2
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    
        # layer 1
        self.fc1 = nn.Linear(input_size, 100) 
        self.batch1 = nn.BatchNorm1d(100)
        self.drop1 = nn.Dropout(0.25)

        # layer 2
        self.fc2 = nn.Linear(100, 80)
        self.batch2 = nn.BatchNorm1d(80)
        self.drop2 = nn.Dropout(0.25)

        # layer 3
        self.fc3 = nn.Linear(80, 60)
        self.batch3 = nn.BatchNorm1d(60)
        self.drop3 = nn.Dropout(0.25)

        # layer 4
        self.fc4 = nn.Linear(60, 40)
        self.batch4 = nn.BatchNorm1d(40)
        self.drop4 = nn.Dropout(0.25)

        self.fc5 = nn.Linear(40, output_size)
        
        self.relu = nn.SELU() 
    def forward(self, x):                            

        # layer 1
        out = self.fc1(x)
        out = self.batch1(out)
        out = self.relu(out)
        out = self.drop1(out)

        # layer 2
        out = self.fc2(out)
        out = self.batch2(out)
        out = self.relu(out)
        out = self.drop2(out)

        # layer 3
        out = self.fc3(out)
        out = self.batch3(out)
        out = self.relu(out)
        out = self.drop3(out)

        # layer 4
        out = self.fc4(out)
        out = self.batch4(out)
        out = self.relu(out)
        out = self.drop4(out)

        out = self.fc5(out)

        return out


In [73]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print('using gpu')

using gpu


In [74]:
#TODO split into real and imaginary parts
X = trnden[140].reshape((10000,4))
y = trntimecoeffs[140]
X.shape, y.shape

((10000, 4), (10000, 2))

In [75]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(X, y, test_size=0.10, random_state=57109, shuffle=True)

In [76]:
train_datat = torch.from_numpy(train_data)
train_labelst = torch.from_numpy(train_labels)
test_datat = torch.from_numpy(test_data)
test_labelst = torch.from_numpy(test_labels)

In [77]:
hidden_size = 100
net = Net(input_size, hidden_size, output_size)
net = net.to(device)

In [78]:
train_datat = train_datat.to(device)
train_labelst = train_labelst.to(device)
test_datat = test_datat.to(device)
test_labelst = test_labelst.to(device)

In [79]:
criterion = nn.CrossEntropyLoss()
criterion2 = nn.MSELoss()
criterion1 = nn.L1Loss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

In [ ]:
num_epochs = 100
batch_size = 256
for epoch in range(1, num_epochs + 1):
        final_score = 0
        final_loss = 0
        for i in range(0, train_datat.shape[0], batch_size):
            
            optimizer.zero_grad()

            if i + batch_size > train_datat.shape[0]:
                batched_data = train_datat[i:]
            else:
                batched_data = train_datat[i: i + batch_size]
            import pdb; pdb.set_trace()
            outputs = net(batched_data)
           
            _, predicted = torch.max(outputs.data, 1)
            train_score = len(predicted[predicted == train_labelst[i: i + batch_size]]) / len(train_labelst[i: i + batch_size])
            
            out = net(test_datat.cfloat())
            _, predictedt = torch.max(out.data, 1)
            test_score = len(predictedt[predictedt == test_labelst]) / len(test_labelst)
            
            final_score = test_score
            loss =  criterion(outputs, train_labelst[i: i + batch_size])
            final_loss = loss
            loss.backward()
            optimizer.step()
            
        losses.append(loss)
        train_acc.append(train_score)
        test_acc.append(test_score)
        
        print('Epoch: {}/{}.............'.format(epoch, num_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))
print('======================================================')


> /tmp/ipykernel_94948/122950603.py(15)<module>()
     13                 batched_data = train_datat[i: i + batch_size]
     14             import pdb; pdb.set_trace()
---> 15             outputs = net(batched_data)
     16 
     17             _, predicted = torch.max(outputs.data, 1)



ipdb>  n


RuntimeError: mat1 and mat2 must have the same dtype
> /tmp/ipykernel_94948/122950603.py(15)<module>()
     13                 batched_data = train_datat[i: i + batch_size]
     14             import pdb; pdb.set_trace()
---> 15             outputs = net(batched_data)
     16 
     17             _, predicted = torch.max(outputs.data, 1)

